In [18]:
import pandas as pd
import numpy as np
import re

In [19]:
df = pd.read_excel('отзывы за лето.xlsx')
df.head(3)

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14


In [20]:
df.isna().sum()

Rating     0
Content    3
Date       0
dtype: int64

In [21]:
df = df.dropna()

In [22]:
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from string import punctuation

exclude = set(punctuation)
sw = set(get_stop_words("ru"))
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in exclude]
    return " ".join(txt)

df['text'] = df['Content'].apply(preprocess_text)
# df = df[df['Rating'] != 3]
# df['target'] = (df['Rating'] > 3)*1

In [23]:
df.Rating.value_counts()

5    14584
1     2276
4     2137
3      911
2      748
Name: Rating, dtype: int64

In [24]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.text, df.Rating, test_size=0.3, random_state=42, stratify=df.Rating)

In [25]:
train_corpus = " ".join(X_train)
train_corpus = train_corpus.lower()

In [26]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")

tokens = word_tokenize(train_corpus)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Отфильтруем данные и соберём в корпус N наиболее частых токенов

In [27]:
tokens_filtered = [word for word in tokens if word.isalnum()]

In [28]:
max_words = 10000
max_len = 100
num_classes = 6

# Training
epochs = 5
batch_size = 512
print_batch_n = 100

In [29]:
from nltk.probability import FreqDist

dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

In [30]:
tokens_filtered_top[:10]

['приложение', 'всё', 'и', 'очень', 'удобно', 'в', 'я', 'на', 'работать', 'с']

In [31]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [32]:
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(str(text).lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [33]:
X_train = np.asarray([text_to_sequence(text, max_len) for text in X_train], dtype=np.int32)
X_test = np.asarray([text_to_sequence(text, max_len) for text in X_test], dtype=np.int32)

In [34]:
X_train

array([[   0,    0,    0, ...,    2,  166,  171],
       [   0,    0,    0, ...,   12, 3752, 2665],
       [   0,    0,    0, ...,    0,    0,    5],
       ...,
       [   0,    0,    0, ...,    0,    0,   28],
       [   0,    0,    0, ...,    4,    5,  180],
       [   0,    0,    0, ...,    5, 2521,    1]], dtype=int32)

## Построение сетей

In [35]:
import keras
from keras.utils import np_utils

y_train = keras.utils.np_utils.to_categorical(y_train, num_classes)
y_test = keras.utils.np_utils.to_categorical(y_test, num_classes)

In [36]:
y_train

array([[0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.],
       ...,
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1.]], dtype=float32)

In [37]:
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard
# from keras.objectives import categorical_crossentropy
from keras.callbacks import EarlyStopping

### tf.keras.layers.Embedding по умолчанию

In [38]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [39]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [40]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    # callbacks=[tensorboard, early_stopping],
                    )

Epoch 1/5
26/26 [==============================] - 12s 27ms/step - loss: 1.4152 - accuracy: 0.6650 - val_loss: 1.0356 - val_accuracy: 0.7026
Epoch 2/5
26/26 [==============================] - 0s 12ms/step - loss: 0.9185 - accuracy: 0.7070 - val_loss: 0.7954 - val_accuracy: 0.7040
Epoch 3/5
26/26 [==============================] - 0s 11ms/step - loss: 0.7499 - accuracy: 0.7571 - val_loss: 0.7246 - val_accuracy: 0.7752
Epoch 4/5
26/26 [==============================] - 0s 12ms/step - loss: 0.6863 - accuracy: 0.7801 - val_loss: 0.6985 - val_accuracy: 0.7690
Epoch 5/5
26/26 [==============================] - 0s 12ms/step - loss: 0.6457 - accuracy: 0.7885 - val_loss: 0.6844 - val_accuracy: 0.7663


In [41]:
preds = model.predict(X_test)

In [42]:
preds = [np.argmax(num) for num in preds]

In [43]:
from sklearn.metrics import classification_report

print(classification_report([np.argmax(l) for l in y_test], preds))

              precision    recall  f1-score   support

           1       0.45      0.71      0.55       683
           2       0.00      0.00      0.00       225
           3       0.00      0.00      0.00       273
           4       0.25      0.11      0.15       641
           5       0.87      0.96      0.91      4375

    accuracy                           0.77      6197
   macro avg       0.31      0.35      0.32      6197
weighted avg       0.69      0.77      0.72      6197



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Модель хорошо предсказывает 5 класс и очень плохо остальные.

### tf.keras.layers.Embedding предобученными векторами

In [44]:
!pip install gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [45]:
from gensim.models import KeyedVectors
import gensim

In [46]:
# taiga_upos_skipgram_300_2_2018

!wget https://rusvectores.org/static/models/rusvectores4/taiga/taiga_upos_skipgram_300_2_2018.vec.gz

!gunzip taiga_upos_skipgram_300_2_2018.vec.gz

--2022-07-24 11:34:51--  https://rusvectores.org/static/models/rusvectores4/taiga/taiga_upos_skipgram_300_2_2018.vec.gz
Resolving rusvectores.org (rusvectores.org)... 116.203.104.23
Connecting to rusvectores.org (rusvectores.org)|116.203.104.23|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 346385366 (330M) [application/x-gzip]
Saving to: ‘taiga_upos_skipgram_300_2_2018.vec.gz’

taiga_upos_skipgram 100%[===================>] 330.34M  19.4MB/s    in 18s     

2022-07-24 11:35:10 (18.3 MB/s) - ‘taiga_upos_skipgram_300_2_2018.vec.gz’ saved [346385366/346385366]



In [47]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['text'])
vocab_size = len(tokenizer.word_index) + 1

In [48]:
embeddings_index = {}
with open('taiga_upos_skipgram_300_2_2018.vec') as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        word = word.split('_')[0]
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 209666 word vectors.


In [49]:
embedding_dim = 300
hits = 0
missed = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        missed += 1

print("Converted %d words (%d missed)" % (hits, missed))

Converted 5672 words (7961 missed)


In [50]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=300, input_length=max_len, weights=[embedding_matrix],
    trainable=False))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [51]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [52]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    # callbacks=[tensorboard, early_stopping],
                    )

Epoch 1/5
26/26 [==============================] - 1s 24ms/step - loss: 1.5634 - accuracy: 0.6383 - val_loss: 1.3108 - val_accuracy: 0.7068
Epoch 2/5
26/26 [==============================] - 0s 14ms/step - loss: 1.1367 - accuracy: 0.7198 - val_loss: 1.0209 - val_accuracy: 0.7137
Epoch 3/5
26/26 [==============================] - 0s 14ms/step - loss: 0.9623 - accuracy: 0.7306 - val_loss: 0.9136 - val_accuracy: 0.7414
Epoch 4/5
26/26 [==============================] - 0s 14ms/step - loss: 0.8840 - accuracy: 0.7465 - val_loss: 0.8556 - val_accuracy: 0.7497
Epoch 5/5
26/26 [==============================] - 0s 14ms/step - loss: 0.8340 - accuracy: 0.7517 - val_loss: 0.8149 - val_accuracy: 0.7628


In [53]:
preds = model.predict(X_test)
preds = [np.argmax(num) for num in preds]
print(classification_report([np.argmax(l) for l in y_test], preds))

              precision    recall  f1-score   support

           1       0.43      0.58      0.50       683
           2       0.00      0.00      0.00       225
           3       0.00      0.00      0.00       273
           4       0.00      0.00      0.00       641
           5       0.80      0.97      0.88      4375

    accuracy                           0.75      6197
   macro avg       0.25      0.31      0.28      6197
weighted avg       0.61      0.75      0.67      6197



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Качество предсказаний модели чуть снизилось при использовании предобученных векторов.